In [24]:
import json
import os

# Verzeichnisse für fixed und random
fixed_path_template = '../07 train models/synth/Llama70B/fixed/split_{}.json'
random_path_template = '../07 train models/synth/Llama70B/random/split_{}.json'

data_dictionary = []

# Füge Daten aus split_0 bis split_4 aus dem "fixed"-Ordner hinzu
for split_index in range(5):
    path_examples = fixed_path_template.format(split_index)
    with open(path_examples, 'r', encoding="utf-8") as file:
        data_dictionary.extend(json.load(file))

# Füge Daten aus split_0 bis split_4 aus dem "random"-Ordner hinzu
for split_index in range(5):
    path_examples = random_path_template.format(split_index)
    with open(path_examples, 'r', encoding="utf-8") as file:
        data_dictionary.extend(json.load(file))

# Beispiel: Drucke die ersten beiden Einträge im kombinierten data_dictionary
len(data_dictionary)

17375

In [25]:
import spacy
nlp = spacy.load("de_core_news_lg")


In [26]:
def get_tags_for_substring(text, start, end):
    doc = nlp(text)
    substring_tags = [(token.pos_, token.idx, token.text) for token in doc if token.idx >= start and token.idx < end]
    return substring_tags

In [27]:
def has_single_word_aspect_term_of_invalid_word_type(ex):
    for tag in [t for t in ex["tags"] if t["type"] == "label-explicit"]:
        part_of_speech_tags_of_aspect_term = get_tags_for_substring(ex["text"], tag["start"], tag["end"])
        if len(part_of_speech_tags_of_aspect_term) == 1:
            if part_of_speech_tags_of_aspect_term[0][0] in ["ADJ", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "PART", "PRON", "SCONJ", "VERB"]:
                return True
    return False

In [28]:
has_single_word_aspect_term_of_invalid_word_type([ex for ex in data_dictionary][0])

False

In [29]:
# Diese analysieren ADJ, ADV, AUX, CONJ, CCONJ, DET, INTJ, PART, PRON, SCONJ, VERB

In [30]:
k = 0
for entry in data_dictionary:
    for tag in [t for t in entry["tags"] if t["type"] == "label-explicit"]:
        potags = get_tags_for_substring(
            entry["text"], tag["start"], tag["end"])
        if len(potags) == 1:
            if potags[0][0] in ["ADJ", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "PART", "PRON", "SCONJ", "VERB"]:
                k += 1
                print(k, tag["text"], potags, entry["llm_prediction_raw"])

        if len(tag["text"]) == 0:
            k += 1
            print(k, tag["text"], potags, entry["llm_prediction_raw"])

1 nicht [('PART', 36, 'nicht')] Sie gaben uns zu verstehen, dass es <aspect-term aspect="FOOD" polarity="NEUTRAL">nicht</aspect-term> wie das Original schmeckt.
2 Servicepersonal [('ADV', 0, 'Servicepersonal')] <aspect-term aspect="SERVICE" polarity="POSITIVE">Servicepersonal</aspect-term> sehr zuvorkommend und hilfsbereit.
3 freundlich [('ADV', 5, 'freundlich')] Sehr <aspect-term aspect="SERVICE" polarity="POSITIVE">freundlich</aspect-term>, aber etwas langsam.
4 neutrales [('ADJ', 57, 'neutrales')] Kleiner <aspect-term aspect="FOOD" polarity="NEGATIVE">Appetizer</aspect-term>, sehr schnell serviert, aber auch sehr <aspect-term aspect="FOOD" polarity="NEUTRAL">neutrales</aspect-term> Geschmackserlebnis.
5 karg [('ADV', 14, 'karg')] Ambiente eher <aspect-term aspect="AMBIENCE" polarity="NEGATIVE">karg</aspect-term>.
6 überteuert [('VERB', 12, 'überteuert')] Einfach nur <aspect-term aspect="PRICE" polarity="NEGATIVE">überteuert</aspect-term>.
7 unfreundlich [('ADV', 25, 'unfreundlich')]